In [228]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Import regular expressions to process emails
import re

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

# library written for this exercise providing additional functions for assignment submission, and others
import utils

# tells matplotlib to embed plots within the notebook
%matplotlib inline

# Import a CSV file
import pandas as pd

In [265]:
musicEmotion = pd.read_csv("Data/data.csv")
musicEmotion = musicEmotion.groupby('track id')

#print(len(musicEmotion.get_group(1)))
#musicEmotion = np.asmatrix(musicEmotion)


48


In [277]:
# musicEmotionOne = musicEmotion.get_group(1)
# musicEmotionOne = np.asmatrix(musicEmotionOne)
# musicEmotionOne[0,2:3]
# print(musicEmotionOne[0,3:4])
print(len(musicEmotion))
print(musicEmotion.get_group(400))

400
      track id  genre   amazement   solemnity   tenderness   nostalgia  \
8395       400    pop           0           0            1           1   
8396       400    pop           0           0            0           1   
8397       400    pop           0           0            0           0   
8398       400    pop           0           1            0           1   
8399       400    pop           0           0            0           0   
8400       400    pop           0           0            0           0   
8401       400    pop           0           0            0           0   
8402       400    pop           1           1            0           0   
8403       400    pop           0           0            0           1   
8404       400    pop           0           0            0           0   
8405       400    pop           1           0            0           0   
8406       400    pop           1           0            0           0   

       calmness   power   joyful_

In [271]:
arrTwo = []
for i in range(1,len(musicEmotion)+1):
    countTwo = 0
    arr = np.zeros(9)
    musicEmotionOne = musicEmotion.get_group(i)
    musicEmotionOne = np.asmatrix(musicEmotionOne)
    for j in range(2,11):
        count = 0
        for m in range(len(musicEmotion.get_group(i))):
            count += musicEmotionOne[m,j:j+1]
        arr[countTwo] = count / len(musicEmotion.get_group(i))
        countTwo += 1
    arrTwo.append(arr)

arrTwo = np.asarray(arrTwo)
print(len(arrTwo))
print(arrTwo[0])

399
[0.14583333 0.33333333 0.20833333 0.29166667 0.625      0.02083333
 0.08333333 0.0625     0.3125    ]


In [276]:
arrTwo = []
for i in range(1,len(musicEmotion)+1):
    countTwo = 0
    arr = np.zeros(9)
    musicEmotionOne = musicEmotion.get_group(i)
    musicEmotionOne = np.asmatrix(musicEmotionOne)
    for j in range(2,11):
        count = np.sum(musicEmotionOne[:len(musicEmotionOne),j:j+1])
        arr[countTwo] = count / len(musicEmotionOne)
        countTwo += 1
    arrTwo.append(arr)
arrTwo = np.asarray(arrTwo)
print(len(arrTwo))
print(arrTwo[399])

400
[0.25       0.16666667 0.08333333 0.33333333 0.08333333 0.
 0.25       0.5        0.41666667]
